https://medium.com/intel-student-ambassadors/music-generation-using-lstms-in-keras-9ded32835a8f

In [7]:
import numpy as np
import pandas as pd
import pydub
from os import path
import os
import glob

In [5]:
!pip3 install pydub

In [8]:
from keras.layers import Dense, LSTM, LeakyReLU
from keras.models import Sequential, load_model
from scipy.io.wavfile import read, write
from tqdm import tqdm

Using TensorFlow backend.


In [25]:
#### mp3 ---> WAV로 변환

from pydub import AudioSegment

mp3_files = glob.glob('music_data/Jazz/*.mp3')

for mp3_file  in mp3_files:
    wav_file = os.path.splitext(mp3_file)[0] + '.wav'
    sound = pydub.AudioSegment.from_mp3(mp3_file)
    sound.export(wav_file, format="wav")
    print(mp3_file)
print("WAV Conversion Complete")



music_data/Jazz/Duke Ellington - Caravan.mp3
music_data/Jazz/Dave Brubeck - Take Five.mp3
music_data/Jazz/The Roy Hargrove Quintet - I'm not so sure.mp3
music_data/Jazz/test.mp3
music_data/Jazz/Roy Hargrove - Strasbourg St. Denis Studio version.mp3
music_data/Jazz/Charlie Parker - Summertime.mp3
music_data/Jazz/Thelonius Monk-Monk's Dream (Take 8) from Monk's Dream LP.mp3
music_data/Jazz/Roy Hargrove Starmaker.mp3
music_data/Jazz/Blue Bossa -Dexter Gordon.mp3
music_data/Jazz/Chick Corea Spain.mp3
music_data/Jazz/Stan Getz & Joao Gilberto - The Girl From Ipanema.mp3
music_data/Jazz/Stan Getz - Misty.mp3
WAV Conversion Complete


In [16]:
wav_files = glob.glob('music_data/Jazz/*.wav')
print(wav_files[:])

["music_data/Jazz/Thelonius Monk-Monk's Dream (Take 8) from Monk's Dream LP.wav", 'music_data/Jazz/Roy Hargrove Starmaker.wav', 'music_data/Jazz/Blue Bossa -Dexter Gordon.wav', 'music_data/Jazz/Chick Corea Spain.wav', 'music_data/Jazz/Stan Getz & Joao Gilberto - The Girl From Ipanema.wav', 'music_data/Jazz/Stan Getz - Misty.wav', 'music_data/Jazz/Duke Ellington - Caravan.wav', 'music_data/Jazz/Dave Brubeck - Take Five.wav', "music_data/Jazz/The Roy Hargrove Quintet - I'm not so sure.wav", "music_data/Jazz/Billie's Bounce Charlie Parker\u3000The Savoy Recordings.wav", 'music_data/Jazz/Roy Hargrove - Strasbourg St. Denis Studio version.wav', 'music_data/Jazz/Charlie Parker - Summertime.wav']


In [18]:
rate, music1 = read(wav_files[0])
rate, music2 = read(wav_files[1])

In [19]:
# taking only some part of the songs and converting to a dataframe
music1 = pd.DataFrame(music1[0:400000, :])
music2 = pd.DataFrame(music2[0:400000, :])

In [20]:
music1.head()

,0,1
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [21]:
music2.tail()

,0,1
399995,371,47
399996,380,11
399997,353,25
399998,360,107
399999,388,100


In [22]:
# function to create train data by shifting the music data 
def create_train_dataset(df, look_back):
    dataX1, dataX2 , dataY1 , dataY2 = [],[],[],[]
    for i in range(len(df)-look_back-1):
        dataX1.append(df.iloc[i : i + look_back, 0].values)
        dataX2.append(df.iloc[i : i + look_back, 1].values)
        dataY1.append(df.iloc[i + look_back, 0])
        dataY2.append(df.iloc[i + look_back, 1])
    return np.array(dataX1), np.array(dataX2), np.array(dataY1), np.array(dataY2)

In [23]:
X1 , X2, y1 , y2 = create_train_dataset(pd.concat([music1.iloc[0 : 160000, :],music2.iloc[0 : 160000, :]], axis=0), 3)

In [24]:
print(X1.shape)
print(X2.shape)
print(y1.shape)
print(y2.shape)

(319996, 3)
(319996, 3)
(319996,)
(319996,)


In [11]:
# function to create train data by shifting the music data
def create_test_data(df, look_back):
    dataX1, dataX2 = [], []
    for i in range(len(df)-look_back-1):
        dataX1.append(df.iloc[i : i + look_back, 0].values)
        dataX2.append(df.iloc[i : i + look_back, 1].values)
        
    return np.array(dataX1), np.array(dataX2)

In [13]:
test1, test2 = create_test_data(pd.concat([music1.iloc[160001 : 400000, :],music2.iloc[160001 : 400000, :]], axis=0), 3)

In [14]:
X1 = X1.reshape((-1, 1, 3))
X2 = X2.reshape((-1, 1, 3))

In [13]:
# LSTM Model for channel 1 of the music data
rnn1 = Sequential()
rnn1.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=50, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=25, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=12, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=1, activation='linear'))
rnn1.add(LeakyReLU())

rnn1.compile(optimizer='adam', loss='mean_squared_error')
rnn1.fit(X1, y1, epochs=20, batch_size=100)





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/20





319996/319996 [==============================] - 24s 76us/step - loss: 57554.1367
Epoch 2/20
319996/319996 [==============================] - 21s 66us/step - loss: 35644.5779
Epoch 3/20
319996/319996 [==============================] - 21s 65us/step - loss: 35157.3996
Epoch 4/20
319996/319996 [==============================] - 20s 63us/step - loss: 34715.7720
Epoch 5/20
319996/319996 [==============================] - 20s 64us/step - loss: 34632.6719
Epoch 6/20
319996/319996 [==============================] - 20s 63us/step - loss: 34248.3890
Epoch 7/20
319996/319996 [==============================] - 21s 65us/step - loss: 34373.5632
Epoch 8/20
319996/319996 [==============================] - 21s 66us/step - loss: 34296.2050
Epoch 9/20
319996/319996 [==============================] - 21s 66us/step - loss: 34175.0817
Epoch 10/20
319996/319996 [==============================

In [15]:
# LSTM Model for channel 2 of the music data
rnn2 = Sequential()
rnn2.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=50, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=25, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=12, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=1, activation='linear'))
rnn2.add(LeakyReLU())

rnn2.compile(optimizer='adam', loss='mean_squared_error')
rnn2.fit(X2, y2, epochs=20, batch_size=100)





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/20





319996/319996 [==============================] - 24s 75us/step - loss: 57669.4946
Epoch 2/20
319996/319996 [==============================] - 20s 63us/step - loss: 39918.4276
Epoch 3/20
319996/319996 [==============================] - 20s 63us/step - loss: 39552.2443
Epoch 4/20
319996/319996 [==============================] - 20s 63us/step - loss: 39492.4526
Epoch 5/20
319996/319996 [==============================] - 20s 63us/step - loss: 39314.4789
Epoch 6/20
319996/319996 [==============================] - 20s 63us/step - loss: 39257.8509
Epoch 7/20
319996/319996 [==============================] - 20s 64us/step - loss: 38789.5924
Epoch 8/20
319996/319996 [==============================] - 20s 63us/step - loss: 38711.2957
Epoch 9/20
319996/319996 [==============================] - 20s 64us/step - loss: 38448.3844
Epoch 10/20
319996/319996 [==============================

In [16]:
# saving LSTM models
rnn1.model.save('rnn1.h5')
rnn2.model.save('rnn2.h5')

NameError: name 'rnn1' is not defined

In [ ]:
# loading the saved models
rnn1 = load_model('rnn1.h5')
rnn2 = load_model('rnn2.h5')

In [ ]:
# making predictions for channel 1 and channel 2
pred_rnn1 = rnn1.predict(test1.reshape(-1, 1, 3))
pred_rnn2 = rnn2.predict(test2.reshape(-1, 1, 3))

In [ ]:
# reshaping the data for input to ANN
X1 = X1.reshape((-1, 3))
X2 = X2.reshape((-1, 3))

In [ ]:
# ANN Model for channel 1 of the music data
ann1 = Sequential()
ann1.add(Dense(units=100, activation='linear', input_dim=3))
ann1.add(LeakyReLU())
ann1.add(Dense(units=50, activation='linear'))
ann1.add(LeakyReLU())
ann1.add(Dense(units=25, activation='linear'))
ann1.add(LeakyReLU())
ann1.add(Dense(units=12, activation='linear'))
ann1.add(LeakyReLU())
ann1.add(Dense(units=1, activation='linear'))
ann1.add(LeakyReLU())

ann1.compile(optimizer='adam', loss='mean_squared_error')
ann1.fit(X1, y1, epochs = 20, batch_size=100)

In [ ]:
# ANN Model for channel 2 of the music data
ann2 = Sequential()
ann2.add(Dense(units=100, activation='linear', input_dim=3))
ann2.add(LeakyReLU())
ann2.add(Dense(units=50, activation='linear'))
ann2.add(LeakyReLU())
ann2.add(Dense(units=25, activation='linear'))
ann2.add(LeakyReLU())
ann2.add(Dense(units=12, activation='linear'))
ann2.add(LeakyReLU())
ann2.add(Dense(units=1, activation='linear'))
ann2.add(LeakyReLU())

ann2.compile(optimizer='adam', loss='mean_squared_error')
ann2.fit(X2, y2, epochs=20, batch_size=100)

In [ ]:
# saving ANN models
ann1.model.save('ann1.h5')
ann2.model.save('ann2.h5')

In [ ]:
# loading saved ANN models
ann1 = load_model('ann1.h5')
ann2 = load_model('ann2.h5')

In [ ]:
# making predictions for channel 1 and channel 2
pred_ann1 = ann1.predict(test1)
pred_ann2 = ann2.predict(test2)

In [ ]:
# saving the ANN predicitons in wav format
write('pred_ann.wav', rate, pd.concat([pd.DataFrame(pred_ann1.astype('int16')), pd.DataFrame(pred_ann2.astype('int16'))], axis=1).values)

In [ ]:
# saving the LSTM predicitons in wav format
write('pred_rnn.wav', rate, pd.concat([pd.DataFrame(pred_rnn1.astype('int16')), pd.DataFrame(pred_rnn2.astype('int16'))], axis=1).values)

In [ ]:
# saving the original music in wav format
write('original.wav',rate, pd.concat([music1.iloc[160001 : 400000, :], music2.iloc[160001 : 400000, :]], axis=0).values)

In [ ]:
Audio("pred_ann.wav")

In [ ]:
Audio("pred_rnn.wav")

In [ ]:
Audio("original.wav")